In [10]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


from matplotlib import rcParams
from wordcloud import WordCloud
from collections import Counter
import csv
from matplotlib import rcParams
import nltk
from nltk.util import ngrams
stop = stopwords.words('english')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score, confusion_matrix,classification_report,plot_confusion_matrix
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SAMBAGA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
print(stopwords.words('indonesian'))

['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasanya', 'baik', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai', 'berdatangan', 'beri', 'berikan', 'berikut', 'berikutnya', 'berjumlah', 'berkali-kali', 'berkata', 'berkehendak', 'berkeinginan'

In [11]:
df_latih = pd.read_csv("Data_latih.csv")
df_uji = pd.read_csv("Data_uji.csv")

In [13]:
df_latih.shape

(4231, 6)

In [14]:
df_uji.shape

(470, 5)

In [15]:
df_latih.head()

,ID,label,tanggal,judul,narasi,nama file gambar
0,71,1,17-Aug-20,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,71.jpg
1,461,1,17-Jul-20,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,461.png
2,495,1,13-Jul-20,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,495.png
3,550,1,8-Jul-20,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",550.png
4,681,1,24-Jun-20,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,681.jpg


In [16]:
df_uji.head()

,ID,tanggal,judul,narasi,nama file gambar
0,238057,13-Jul-20,Narasi Tito Karnavian Berideologi Komunis Kare...,TITO KARNIVAN ITU BERIDIOLOGI KOMUNIS DIA BISA...,238057.jpg
1,238158,6-Jul-20,Anies: Seberat beratnya Pekerjaan Akan terasa ...,Seberat beratnya Pekerjaan Akan terasa ringan ...,238158.jpg
2,238865,22-Apr-20,Hindu di india Melemparkan Patung Buatan Merek...,Hindu di india melemparkan patung buatan merek...,238865.jpg
3,248298,22-Oct-19,RSCM Praktekkan Penyedotan Plug Vena/Saluran ...,Mulai Hari ini di RSCM mulai diPraktekkan Peny...,248298.jpg
4,255176,1-May-20,Permohonan Kelonggaran Angsuran ke OJK,"Untuk sekedar info, Bagi anda yg punya ansuran...",255176.jpg


In [17]:
df_latih.isnull().sum()

ID                  0
label               0
tanggal             0
judul               0
narasi              0
nama file gambar    0
dtype: int64

In [18]:
X = df_latih.drop(columns="label", axis=1)
Y = df_latih["label"]

In [19]:
X

,ID,tanggal,judul,narasi,nama file gambar
0,71,17-Aug-20,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,71.jpg
1,461,17-Jul-20,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,461.png
2,495,13-Jul-20,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,495.png
3,550,8-Jul-20,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",550.png
4,681,24-Jun-20,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,681.jpg
...,...,...,...,...,...
4226,999287,17-Jan-16,: KPK Dilarang Membawa Brimob Bersenjata Masuk...,KPK Dilarang Membawa Brimob Bersenjata Masuk G...,999287.jpg
4227,999472,20-Aug-20,Foto pejabat keuangan dibawah palu arit,Jangan mau dialihkan kepada pakaian adat… FOKU...,999472.png
4228,999501,17-Aug-20,Gambar Denny Siregar Musuh Warga Tasikmalaya d...,Akhir nya bisa terkenal bang denny,999501.jpg
4229,999848,21-Jul-20,Kaesang: Bapak Saya dengan Kesederhaan Bisa Ni...,Bapak Saya dengan Kesederhaan Bisa Nipu Rakyat...,999848.jpg


In [20]:
Y

0       1
1       1
2       1
3       1
4       1
       ..
4226    1
4227    1
4228    1
4229    1
4230    1
Name: label, Length: 4231, dtype: int64

In [22]:
port_stem = PorterStemmer()

In [23]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('indonesian')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [24]:
df_latih['judul'] = df_latih['judul'].apply(stemming)

In [25]:
X = df_latih["judul"].values
Y = df_latih["label"].values

In [26]:
X

array(['pemakaian masker menyebabkan penyakit legionnair',
       'instruksi gubernur jateng penilangan yg bermask muka rp e tilang via app pikobar',
       'foto jim rohn jokowi presiden terbaik dlm sejarah bangsa indonesia',
       ..., 'gambar denni siregar musuh warga tasikmalaya truk',
       'kaesang kesederhaan nipu rakyat indonesia',
       'laser termomet gun merusak struktur otak'], dtype=object)

In [27]:
Y

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [28]:
X.shape

(4231,)

In [29]:
Y.shape

(4231,)

In [31]:
from sklearn.feature_extraction.text import TfidfTransformer

x_train,x_test,y_train,y_test = train_test_split(X, Y, test_size=0.2, random_state=2020)

pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LogisticRegression())])

model = pipe.fit(x_train, y_train)
prediction = model.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))


accuracy: 82.88%


In [32]:
from sklearn.metrics import f1_score

print(f'F1 Score Valid Data : {f1_score(y_test, prediction)}')

F1 Score Valid Data : 0.9042904290429044


In [62]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [63]:
X

<4231x8825 sparse matrix of type '<class 'numpy.float64'>'
	with 36376 stored elements in Compressed Sparse Row format>

In [26]:
print(X)

  (0, 6243)	0.4251289595284841
  (0, 5946)	0.48040000082830064
  (0, 5087)	0.4314843500281596
  (0, 4546)	0.393121240843696
  (0, 4202)	0.49775310411155316
  (1, 8769)	0.1863296648440099
  (1, 8584)	0.2680928333481104
  (1, 8488)	0.23500072208228154
  (1, 8255)	0.23243414972675577
  (1, 8244)	0.13881188998064656
  (1, 8030)	0.20380389158881
  (1, 6979)	0.1869893586558044
  (1, 6476)	0.2857841948352721
  (1, 6208)	0.2857841948352721
  (1, 5346)	0.2857841948352721
  (1, 4923)	0.20070036463310192
  (1, 3203)	0.2564398314399943
  (1, 3052)	0.2758209367285257
  (1, 2632)	0.20380389158881
  (1, 1644)	0.08129598876902022
  (1, 871)	0.2857841948352721
  (1, 475)	0.20380389158881
  (1, 311)	0.2998266184417643
  (2, 8050)	0.3342336179233318
  (2, 7246)	0.3531789805291117
  :	:
  (4228, 7541)	0.3860040614887939
  (4228, 5391)	0.3860040614887939
  (4228, 2468)	0.2934527541853932
  (4228, 1644)	0.11377157303499132
  (4228, 1608)	0.3860040614887939
  (4228, 683)	0.35240874871078814
  (4229, 8244)	0.

In [64]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [65]:
model = LogisticRegression()

In [66]:
model.fit(X_train, Y_train)

LogisticRegression()

In [67]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.8525413711583925


In [68]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the training data : ', test_data_accuracy)

Accuracy score of the training data :  0.8276269185360094


In [76]:
news = "Sandi Sarankan Warga Ngadu ke Kelurahan Saja"
news = [news]
news = [stemming(news[0])]
news = vectorizer.transform(news)
predict = model.predict(news)


print(predict)

if (predict[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake
